In [1]:
import os

In [2]:
%pwd

'c:\\Users\\INPUNPRAP\\OneDrive - Sandvik\\Desktop\\Transfer_learning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\INPUNPRAP\\OneDrive - Sandvik\\Desktop\\Transfer_learning'

In [6]:
#Step 1: Update config.yaml file :  All fixed path / configuration settings for your deep learning model or experiment.which is a human-readable data serialization format.ike the learning rate, batch size, model architecture, optimization algorithm, or paths to data files.

In [7]:
# Step2: Secret.yaml file for database credentials etc..

In [8]:
# Step3: Update param.yaml file: model parameters are kept here

In [34]:
# Step 4: update the entity: return type of a function . often used to define classes or structures that represent entities or objects relevant ,Dataset,Model,Training example,Evaluation metrics,Configuration.may define classes or data structures specific to these entities, along with any associated methods or functions that operate on or interact with them. By encapsulating the entities and their related logic within a dedicated file, you can improve code organization, readability, and maintainability.
from dataclasses import dataclass
from pathlib import Path

In [35]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path

In [37]:
# Step5: Configuration.py file code: ensures that any changes made to the configuration files are properly reflected and accessible within your project
from ChipClassifier.constants import *
from ChipClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])          

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

NameError: name 'CONFIG_FILE_PATH' is not defined

In [ ]:
# step6: update the components

import os
import urllib.request as request 
import zipfile
from ChipClassifier import logger
from ChipClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(url=self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f"{filename} download: with following info: \n {headers}")

        else:
            logger.info(f"File already exists of size: {get_size(path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path : str
        Extract zip file into data directory 
        Function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
# create pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e